# KLUE 데이터셋 확인

In [3]:
from datasets import load_dataset

klue = load_dataset('klue', 're')

Reusing dataset klue (/opt/ml/.cache/huggingface/datasets/klue/re/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
klue['train'][0]

{'guid': 'klue-re-v1_train_00000',
 'sentence': '〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.',
 'subject_entity': {'word': '비틀즈',
  'start_idx': 24,
  'end_idx': 26,
  'type': 'ORG'},
 'object_entity': {'word': '조지 해리슨',
  'start_idx': 13,
  'end_idx': 18,
  'type': 'PER'},
 'label': 0,
 'source': 'wikipedia'}

In [5]:
label_dict = {
    "x": 0,
    "no_relation": 1,
    "role": 2,
    "title": 3,
    "colleagues": 4,
    "enemy": 5,
    "family": 6,
    "alternate_names": 7,
    "open": 8,
    "possess": 9,
    "production": 10,
    "employee_of": 11,
    "associate_with": 12,
    "cause_of_death": 13,
}

In [6]:
import pandas as pd

tagging_file = pd.read_csv('메인 태깅.csv')

In [8]:
tagging_file.head(1)

,id,sub_entity,obj_entity,sub_index,obj_index,relation,sub_text,obj_text,sentence,ann1 동규,ann2 석민,ann3 태현
0,석민_00299,SUBJ,OBJ,21,10,no_relation,'어벤져스: 인피니티 워 제1부',2014년 10월,"이 영화의 제목은 2014년 10월, '어벤져스: 인피니티 워 제1부'로 결정, 발...",no_relation,no_relation,no_relation


In [7]:
re_dataset_dict = {
    "guid": [],
    "sentence": [],
    "subject_entity": [],
    "object_entity": [],
    "label": [],
    "source": [],
}

In [ ]:
def freq_label(tags):
  voting_dict = {}
  for idx, tag in enumerate(tags):
    score = 1 if idx != 1 else 1.01 # 동규님꺼에 점수를 더줌
    if tag not in voting_dict.keys():
      voting_dict[tag] = score
    else:
      voting_dict[tag] += score
  for key in voting_dict.keys():
    score = 0
    new_answer = ''
    for answer in voting_dict[key].keys():
      if voting_dict[key][answer] > score:
        score = voting_dict[key][answer]
        new_answer = answer
  return int(new_answer)

In [ ]:
for idx, tag in tagging_file.iterrows():
  tags = [label_dict[tag['relation']], label_dict[tag['ann1 동규']], label_dict[tag['ann2 석민']], label_dict[tag['ann3 태현']]]
  if 0 in tags:
    continue
  label = freq_label(tags)
  re_dataset_dict["guid"].append(f"{idx:05d}")
  re_dataset_dict["sentence"].append(tag['sentence'])
  re_dataset_dict["source"].append('wikipedia')
  re_dataset_dict["label"].append(label)
  re_dataset_dict["subject_entity"].append({
    'word': tag["sub_text"],
    'start_idx': tag["sub_index"],
    'end_idx': tag["sub_index"] + len(tag["sub_text"]) - 1,
    'type': tag["sub_entity"]
  })
  re_dataset_dict["object_entity"].append({
    'word': tag["obj_text"],
    'start_idx': tag["obj_index"],
    'end_idx': tag["obj_index"] + len(tag["obj_text"]) - 1,
    'type': tag["obj_entity"]
  })

In [ ]:
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_dict(re_dataset_dict)

re_dict = {
  'train': train_dataset
}

re_datasets = DatasetDict(re_dict)
re_datasets.save_to_disk('./re_datasets')